In [ ]:
SANDBOX_NAME = # Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"



# Agrupando

Podemos agrupar información de nuestras estructuras de datos de forma muy sencilla mediante el método `groupby`. Normalmente se sigue una estrategia de separar-aplicar-combinar (*split-apply-combine*). Lo que se hace es separar los datos iniciales en grupos de interés, sobre cada grupo se aplica cierta funcionalidad y el resultado se combina en una nueva estructura de datos.

In [ ]:
import pandas as pd



swim100m.csv Source: ‘Statistical Modeling: A Fresh Approach’

In [ ]:
data = spark.read.csv(DATA_PATH+'swim100m.csv', header=True, inferSchema=True).toPandas()

#Local reading

# data = pd.read_csv('datasets/swim100m.csv')
data.head()



Agrupamos por sexo

In [ ]:
grouped_sex = data.groupby(by='sex')
grouped_sex



El número de agrupaciones por sexo (len(grouped_sex)) es igual al número de elementos únicos de esa categoria

In [ ]:
data['sex'].nunique() == len(grouped_sex)



Números de elementos de cada grupo

In [ ]:
grouped_sex.size()



Devuelve el primer elemento de cada grupo

In [ ]:
grouped_sex.first()



Devuelve un diccionario con los grupos creados y las rtiquetas (registros) que pertecen a cada grupo.

In [ ]:
grouped_sex.groups



Mediante get_group podemos seleccionar uno de los grupos y devolvernos su información como un dataframe

In [ ]:
grouped_sex.get_group('M').head()



Una vez realizada la agrupación que nos interesa podemos aplicar una transformación sobre alguna de las variables restantes.



En este caso calculamos la media del año por sexo

In [ ]:
a = data.groupby('sex')['year'].mean()
print(a)



Se puede iterar sobre el resultado obtenido con `groupby` (devuelve una tupla). El primer item es el valor de las columnas y el segundo es el `DataFrame` filtrado:

In [ ]:
for k, v in data.groupby('sex'):
    print(k, ':', v.mean(), '\n')



También se pueden agrupar los datos respecto a más de una columna. El resultado son tuplas anidadas, por ejemplo:

In [ ]:
auto = spark.read.csv(DATA_PATH+'Auto.csv', header=True, inferSchema=True).toPandas()

#Local reading

# auto = pd.read_csv('../../data/Auto.csv')
auto.head()

In [ ]:
auto.groupby(['horsepower', 'cylinders']).mean()



Otra opción para trabajar con datos agrupados en `Pandas` es utilizar la función agg(), que nos permite aplicar varias funciones sobre una agrupación.

In [ ]:
fortune = spark.read.csv(DATA_PATH+'fortune1000.csv', header=True,
                         inferSchema=True).toPandas().set_index('Rank', inplace=True)

#Local reading

# fortune = pd.read_csv('../../data/fortune1000.csv', index_col='Rank')

In [ ]:
fortune.head()



Agrupamos por sector y realizamos operaciones sobre cada grupo y varios campos.

In [ ]:
sectors = fortune.groupby(by='Sector')
sectors.agg({'Revenue':['sum','mean'], 'Profits':'sum','Employees':'mean'}) 



Otra forma de agrupar la información de un dataframe es a través de anidar índices, dando lugar a una **multiIndex dataframe**

In [ ]:
bigmac = spark.read.csv(DATA_PATH+'bigmac.csv', 
                        header=True, inferSchema=True).toPandas()
bigmac['Date'] = pd.to_datetime(bigmac['Date'], infer_datetime_format=True)

#Local reading

# bigmac = pd.read_csv('../../data/bigmac.csv', parse_dates= ['Date'])
bigmac.head(3)

In [ ]:
bigmac.set_index(keys = ['Date','Country'], inplace=True) # Creates a multindex DF
bigmac.sort_index(inplace= True)
bigmac



El acceso a los datos se realiza pasando la tupla de valores correspondiente al índice

In [ ]:
bigmac.loc[('2010-01-01','Argentina'),'Price in US Dollars']



# Ejercicios



## Ejercicio 1 

Dado el fichero de datos datasets/u.user



- a. Cargue el fichero en un dataframe (indexe por 'user_id') y estudie las estructura de los datos.
- b. ¿Cúal es la edad media por profesión?.
- c. Para cada profesión, ¿cúal es la edad mínima y máxima?.
- d. Para cada combinación de profesión y género, calcule la edad media.
- e. Para cada profesión, calcule el porcentaje de mujeres y hombres



**a.** Cargue el fichero en un dataframe (indexe por 'user_id') y estudie las estructura de los datos.

In [ ]:
# Respuesta

users = spark.read.csv(DATA_PATH+'u.user', sep='|').toPandas().set_index('user_id', inplace=True)

#Local reading

# users = pd.read_table('datasets/u.user', sep='|', index_col='user_id')
users.head()



**b.** ¿Cúal es la edad media por profesión?

In [ ]:
# Respuesta

users.groupby('occupation').age.mean()



**c.** Para cada profesión, ¿cúal es la edad mínima y máxima?

In [ ]:
# Respuesta

users.groupby('occupation').age.agg(['min', 'max'])



**d.** Para cada combinación de profesión y género, calcule la edad media

In [ ]:
# Respuesta

users.groupby(['occupation', 'gender']).age.mean()



**e**. Para cada profesión, calcule el porcentaje de mujeres y hombres.



Calculamos el total por profesiones

In [ ]:
# Respuesta

occup_count = users.groupby(['occupation']).agg('count')



Creamos un DF agrupado por occupation/gender y calculamos el número por gender

In [ ]:
# Respuesta

gender_ocup = users.groupby(['occupation', 'gender']).agg({'gender': 'count'})



Dividimos el segundo DF (occupation/gender/count) entre el primero (count por profesion)

In [ ]:
# Respuesta

occup_gender = gender_ocup.div(occup_count, level = "occupation") * 100

occup_gender.loc[: , 'gender']



## Ejercicio 2 

Dados los siguientes datos:

raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}



- a. Cree un dataframe con nombre 'regiment'. No olvide asignar el nombre correspondiente a cada columna.
- b. Muestre un resumen estadístico para cada compañia
- c. Muestre la media de preTestScores agrupado por regiment y company
- d. Muestre la media de preTestScores agrupado por regiment y company sin usar un índice jerárquico, es decir, que cada company represente una columna.
- e. ¿Cúal es el número de observaciones por regiment y company?
- f. Agrupe sobre regiment, itere sobre los grupos creados e imprima la información de cada uno.





**a.** Cree un dataframe con nombre 'regiment'. No olvide asignar el nombre correspondiente a cada columna.

In [ ]:
# Respuesta

raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}

In [ ]:
# Respuesta

regiment = pd.DataFrame(raw_data, columns = raw_data.keys())
regiment



**b.** Muestre un resumen estadístico para cada compañia

In [ ]:
# Respuesta

regiment.groupby('company').describe()



**c.** Muestre la media de preTestScores agrupado por regiment y company

In [ ]:
# Respuesta

regiment.groupby(['regiment', 'company']).preTestScore.mean()



**d.** Muestre la media de preTestScores agrupado por regiment y company sin usar un índice jerárquico, es decir, que cada company represente una columna.

In [ ]:
# Respuesta

regiment.groupby(['regiment', 'company']).preTestScore.mean().unstack()



**e.** ¿Cúal es el número de observaciones por regiment y company?

In [ ]:
# Respuesta

regiment.groupby(['company', 'regiment']).size()



**f.** Agrupe sobre regiment, itere sobre los grupos creados e imprima la información de cada uno.

In [ ]:
# Respuesta

for name, group in regiment.groupby('regiment'):
    # We print each regiment name
    print('\n',name,'\n')
    # We print each regiment datas
    print(group)